In [1]:
import warnings
import numpy as np 
# Ignore all warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd 
import matplotlib.pyplot as plt 

In [3]:
# Importing the energy data 
df_energy = pd.read_csv('Data/energy_dataset.csv',parse_dates=['time'])

In [4]:
# To see the general information of all column 
pd.set_option('display.max_columns',None)
df_energy.describe()

,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,generation hydro pumped storage aggregated,generation hydro pumped storage consumption,generation hydro run-of-river and poundage,generation hydro water reservoir,generation marine,generation nuclear,generation other,generation other renewable,generation solar,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,forecast wind offshore eday ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
count,35045.000000,35046.000000,35046.0,35046.000000,35046.000000,35045.000000,35046.0,35046.0,35046.0,0.0,35045.000000,35045.000000,35046.000000,35045.0,35047.000000,35046.000000,35046.000000,35046.000000,35045.000000,35046.0,35046.000000,35064.000000,0.0,35064.000000,35064.000000,35028.000000,35064.000000,35064.000000
mean,383.513540,448.059208,0.0,5622.737488,4256.065742,298.319789,0.0,0.0,0.0,NaN,475.577343,972.116108,2605.114735,0.0,6263.907039,60.228585,85.639702,1432.665925,269.452133,0.0,5464.479769,1439.066735,NaN,5471.216689,28712.129962,28696.939905,49.874341,57.884023
std,85.353943,354.568590,0.0,2201.830478,1961.601013,52.520673,0.0,0.0,0.0,NaN,792.406614,400.777536,1835.199745,0.0,839.667958,20.238381,14.077554,1680.119887,50.195536,0.0,3213.691587,1677.703355,NaN,3176.312853,4594.100854,4574.987950,14.618900,14.204083
min,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,NaN,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,NaN,237.000000,18105.000000,18041.000000,2.060000,9.330000
25%,333.000000,0.000000,0.0,4126.000000,2527.000000,263.000000,0.0,0.0,0.0,NaN,0.000000,637.000000,1077.250000,0.0,5760.000000,53.000000,73.000000,71.000000,240.000000,0.0,2933.000000,69.000000,NaN,2979.000000,24793.750000,24807.750000,41.490000,49.347500
50%,367.000000,509.000000,0.0,4969.000000,4474.000000,300.000000,0.0,0.0,0.0,NaN,68.000000,906.000000,2164.000000,0.0,6566.000000,57.000000,88.000000,616.000000,279.000000,0.0,4849.000000,576.000000,NaN,4855.000000,28906.000000,28901.000000,50.520000,58.020000
75%,433.000000,757.000000,0.0,6429.000000,5838.750000,330.000000,0.0,0.0,0.0,NaN,616.000000,1250.000000,3757.000000,0.0,7025.000000,80.000000,97.000000,2578.000000,310.000000,0.0,7398.000000,2636.000000,NaN,7353.000000,32263.250000,32192.000000,60.530000,68.010000
max,592.000000,999.000000,0.0,20034.000000,8359.000000,449.000000,0.0,0.0,0.0,NaN,4523.000000,2000.000000,9728.000000,0.0,7117.000000,106.000000,119.000000,5792.000000,357.000000,0.0,17436.000000,5836.000000,NaN,17430.000000,41390.000000,41015.000000,101.990000,116.800000


In [5]:
# Delete column which contains only '0' value 
columns_zero = df_energy.columns[(df_energy.min() == 0) & (df_energy.max()==0)]

df_energy.drop(columns = columns_zero, inplace = True)

In [6]:
# And also drop columns which contains only NA value 
df_energy.dropna(axis = 1, how='all',inplace=True)

In [7]:
# We can see that now several columns have small amount of NA values
df_energy.isna().sum()

time                                            0
generation biomass                             19
generation fossil brown coal/lignite           18
generation fossil gas                          18
generation fossil hard coal                    18
generation fossil oil                          19
generation hydro pumped storage consumption    19
generation hydro run-of-river and poundage     19
generation hydro water reservoir               18
generation nuclear                             17
generation other                               18
generation other renewable                     18
generation solar                               18
generation waste                               19
generation wind onshore                        18
forecast solar day ahead                        0
forecast wind onshore day ahead                 0
total load forecast                             0
total load actual                              36
price day ahead                                 0


In [8]:
# For these NA values, we should interpolate
df_energy = df_energy.fillna(method='ffill')

In [9]:
# And we will make our row index the time information
df_energy['time'] = pd.to_datetime(df_energy['time'],utc=True, infer_datetime_format=True)
df_energy = df_energy.set_index('time')

## Weather

In [10]:
# Importing the weather data 
df_weather = pd.read_csv('Data\weather_features.csv', parse_dates=['dt_iso'])

In [11]:
# Thanksfully, there is no NA value.
df_weather.isna().sum(axis=0)

dt_iso                 0
city_name              0
temp                   0
temp_min               0
temp_max               0
pressure               0
humidity               0
wind_speed             0
wind_deg               0
rain_1h                0
rain_3h                0
snow_3h                0
clouds_all             0
weather_id             0
weather_main           0
weather_description    0
weather_icon           0
dtype: int64

In [12]:
df_weather['weather_description'].value_counts()

sky is clear                    82685
few clouds                      32101
broken clouds                   17448
scattered clouds                15945
light rain                      10905
mist                             3908
moderate rain                    3621
overcast clouds                  2561
fog                              2506
light intensity drizzle          1241
heavy intensity rain             1142
light intensity shower rain       657
proximity thunderstorm            480
proximity shower rain             476
haze                              435
shower rain                       426
drizzle                           369
dust                              345
thunderstorm                      309
light snow                        140
thunderstorm with rain            127
thunderstorm with light rain       92
heavy intensity shower rain        81
light intensity drizzle rain       79
very heavy rain                    78
snow                               74
smoke       

In [13]:
values_counts = df_weather['weather_description'].value_counts()

In [14]:
top_5_values = values_counts.head(5).index.tolist()
df_weather.loc[~df_weather['weather_description'].isin(top_5_values),'weather_description'] = 'others'

In [15]:
df_weather['weather_description'].value_counts()

sky is clear        82685
few clouds          32101
others              19312
broken clouds       17448
scattered clouds    15945
light rain          10905
Name: weather_description, dtype: int64

In [16]:
# Drop the variable relevant to weather information beside 'weather_description'
columns_drop_weather = ['weather_id','weather_main','weather_icon']
df_weather.drop(columns=columns_drop_weather, inplace=True)

In [17]:
dummy_df = pd.get_dummies(df_weather['weather_description'],prefix='weather')
dummy_df = dummy_df.apply(lambda x: x.astype(bool).astype(int))

In [18]:
df_weather = pd.concat([df_weather.loc[:,df_weather.columns !='weather_description'],dummy_df],axis=1)

In [19]:
# Also temp_min and temp_max are mostly duplicate with 'temp' variable 
df_weather = df_weather.drop(columns = ['temp_min','temp_max','rain_3h','snow_3h'])

In [20]:
# Now we are going to change the int64 to float64 for being in line with df_energy

def df_convert_dtypes(df, convert_from, convert_to):
    cols = df.select_dtypes(include=[convert_from]).columns
    for col in cols:
        df[col] = df[col].values.astype(convert_to)
    return df
df_weather = df_convert_dtypes(df_weather, np.int64, np.float64)

In [21]:
df_weather['time'] = pd.to_datetime(df_weather['dt_iso'], utc=True, infer_datetime_format=True)
df_weather = df_weather.drop(['dt_iso'], axis=1)
df_weather = df_weather.set_index('time')

In [22]:
# Now let's see how many the number of rows for each city. 
# The each number has to be same with the number of energy dataset row
cities = df_weather['city_name'].unique()
grouped_weather = df_weather.groupby('city_name')
for city in cities:
    print('There are {} observations in df_weather'
          .format(grouped_weather.get_group('{}'.format(city)).shape[0]), 
          'about city: {}.'.format(city))
print('There are {} observations in df_energy.'.format(df_energy.shape[0]))
# Unfortunately, quite different. 

There are 35145 observations in df_weather about city: Valencia.
There are 36267 observations in df_weather about city: Madrid.
There are 35951 observations in df_weather about city: Bilbao.
There are 35476 observations in df_weather about city:  Barcelona.
There are 35557 observations in df_weather about city: Seville.
There are 35064 observations in df_energy.


In [23]:
# So we have to erase that 
df_weather = df_weather.reset_index().drop_duplicates(subset=['time', 'city_name'],
                                                      keep='first').set_index('time')

In [24]:
# Now let's see how many observations each city has.
print('There are {} observations in df_energy.'.format(df_energy.shape[0]))

grouped_weather = df_weather.groupby('city_name')

for city in cities:
    print('There are {} observations in df_weather'
          .format(grouped_weather.get_group('{}'.format(city)).shape[0]), 
          'about city: {}.'.format(city))

There are 35064 observations in df_energy.
There are 35064 observations in df_weather about city: Valencia.
There are 35064 observations in df_weather about city: Madrid.
There are 35064 observations in df_weather about city: Bilbao.
There are 35064 observations in df_weather about city:  Barcelona.
There are 35064 observations in df_weather about city: Seville.


In [25]:
# Below is the process for merging energy dataset with weather information of each city
df_1, df_2, df_3, df_4, df_5 = [x for _, x in df_weather.groupby('city_name')]
dfs = [df_1, df_2, df_3, df_4, df_5]

df_final = df_energy

for df in dfs:
    city = df['city_name'].unique()
    city_str = str(city).replace("'", "").replace('[', '').replace(']', '').replace(' ', '')
    df = df.add_suffix('_{}'.format(city_str))
    df_final = df_final.merge(df, on=['time'], how='outer')
    df_final = df_final.drop('city_name_{}'.format(city_str), axis=1)


In [26]:
for i in range(len(df_final)):
    position = df_final.index[i]
    hour = position.hour
    weekday = position.weekday()
    month = position.month
    df_final.loc[position, 'hour'] = hour
    df_final.loc[position, 'weekday'] = weekday
    df_final.loc[position, 'month'] = month

In [27]:
# From here, I will use df_final_dummy
df_final_dummy = df_final.copy()

In [28]:
df_final_dummy['hour'] = df_final_dummy['hour'].astype('category')

In [29]:
dummy_variables_hours = pd.get_dummies(df_final_dummy['hour'])
dummy_variables_hours.columns = ['hour_' + str(col) for col in dummy_variables_hours.columns]



In [30]:
# Concatenate the dummy variables with the original DataFrame
df_final_dummy = pd.concat([df_final_dummy, dummy_variables_hours], axis=1)

# Optionally, drop the original column if you no longer need it
df_final_dummy.drop('hour', axis=1, inplace=True)

In [31]:
df_final_dummy['month'] = df_final_dummy['month'].astype('category')

In [32]:
dummy_variables_months = pd.get_dummies(df_final_dummy['month'])
dummy_variables_months.columns = ['month_' + str(col) for col in dummy_variables_months.columns]


In [33]:
# Concatenate the dummy variables with the original DataFrame
df_final_dummy = pd.concat([df_final_dummy, dummy_variables_months], axis=1)

# Optionally, drop the original column if you no longer need it
df_final_dummy.drop('month', axis=1, inplace=True)

In [34]:
df_final_dummy['weekday_dummy'] = df_final_dummy['weekday'].apply(lambda x: 1 if x <= 5 else 0)

In [35]:
df_final_dummy.drop('weekday', axis=1, inplace=True)

In [36]:
df_final_dummy.reset_index(inplace=True)

In [144]:
df_final_dummy.to_csv('df_final.csv', index=False)